In [1]:
import abem as ab
import numpy as np
from numpy.linalg import norm


frequency = 20.0 # frequency [Hz]
k = ab.frequency_to_wavenumber(frequency)

interior_points = np.array([[0.500, 0.000, 0.000],
                           [0.000, 0.000, 0.010],
                           [0.000, 0.000, 0.250],
                           [0.000, 0.000, 0.500],
                           [0.000, 0.000, 0.750]], dtype=np.float32)

# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#
solver = ab.InteriorHelmholtzSolver3D(ab.sphere())

boundary_condition = solver.dirichlet_boundary_condition()
boundary_condition.f[:] = np.sin(k * solver.centers[:,2])

boundary_incidence = ab.BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

interior_incident_phi = np.zeros(interior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("Test Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# Neumann boundary condition with v = cos(k*z)
#
boundary_condition = solver.neumann_boundary_condition()

for i in range(solver.len()):
    a, b, c = solver.geometry.triangle_vertices(i)
    normal = ab.normal_3d(a, b, c)
    boundary_condition.f[i] = k * np.cos(k * solver.centers[i,2]) * normal[2]

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Neumann boundary condition with v = cos(k*z)
#
boundary_condition = solver.neumann_boundary_condition()

p = np.array([0.0, 0.0, 0.0], dtype=np.float32)
for i in range(solver.len()):
    q = solver.centers[i, :]
    r = p - q
    R = norm(r)
    boundary_incidence.phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)
    a, b, c = solver.geometry.triangle_vertices(i)
    normal = ab.normal_3d(a, b, c)
    drbdn = -np.dot(r, normal) / R
    boundary_incidence.v[i] = drbdn * np.exp(1.0j * k * R)*(1.0j * k * R - 1.0) / \
            (4.0 * np.pi * R * R)

for i in range(interior_points.shape[0]):
    q = interior_points[i, :]
    r = p - q
    R = norm(r)
    interior_incident_phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)

Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 0.36530147134765034 (20.0 Hz)

index   Potential               Pressure                 Velocity                 Intensity

    1   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3805e-01-5.1565e-03i  -1.0947e-01
    2   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3799e-01-5.1442e-03i  -1.0921e-01
    3   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3805e-01-5.1565e-03i  -1.0947e-01
    4   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3805e-01-5.1565e-03i  -1.0947e-01
    5   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3799e-01-5.1442e-03i  -1.0921e-01
    6   2.8039e-01+0.0000e+00   0.0000e+00+4.2459e+01i   3.3805e-01-5.1565e-03i  -1.0947e-01
    7   1.6172e-01+0.0000e+00   0.0000e+00+2.4489e+01i   1.8979e-01-7.3123e-03i  -8.9536e-02
    8   8.1129e-02+0.0000e+00   0.0000e+00+1.2285e+01i   9.1398e-02-7.8868e-03i  -4.8444e-02
    9   1.6172e-01+0.